# TCC - Estudo da base de alvarás de Curitiba

Estrutura básica do código:
 - Importações das bibliotecas
 - Definições das funções
 - Programa em si

<b>Resumo do funcionamento:</b><br>
Leitura das tabelas que estão no banco SQL chamado BIGSEA, contendo os dados já tratados sobre a cidade de Curitiba e posterior tratamento e análise exploratória feitos em Python.

<b>Instrução prévia, antes de rodar este programa:</b><br>
Abrir o <b>PuTTy</b> e rodar a conexão <b>C3SL</b>, para estabelecer a conexão com o banco SQL BIGSEA.

# Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np

import datetime as dt

import geopandas as gpd

import psycopg2

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Definição de funções e variáveis globais

In [3]:
def Conecta_Pgadmin(db_name):
    conn = psycopg2.connect(host = 'localhost', database = db_name, port = '5435', user = 'postread', password = 'PostRead')
    return conn

In [4]:
def Base_Alvaras():
    nome_tabela = "prefeitura.alvara"
    postgreSQL_select_Query = "SELECT * FROM " + nome_tabela
    df_alvaras_raw = pd.read_sql(postgreSQL_select_Query, Conecta_Pgadmin('postgiscwb'))
    return df_alvaras_raw

In [5]:
def Mapa_Curitiba_Bairros():
    gdf_mapa_bairros = gpd.GeoDataFrame.from_postgis("SELECT * FROM limites_legais.divisa_de_bairros", Conecta_Pgadmin('bigsea'), geom_col='geom', crs=5533).to_crs(4326)
    return gdf_mapa_bairros

In [6]:
def Corrige_Nomes_Bairros(nome_corrigir):
    dict_correcoes = {'CIDADE INDUSTRIAL':'CIDADE INDUSTRIAL DE CURITIBA', 
'CIC':'CIDADE INDUSTRIAL DE CURITIBA', 
'SITIO CERCADO':'SÍTIO CERCADO', 
'MARUMBI II':'UBERABA', 
'SANTA CANDIDA':'SANTA CÂNDIDA', 
'JARDIM BOTANICO':'JARDIM BOTÂNICO', 
'AUTÓDROMO':'CAJURU', 
'TARUMA':'TARUMÃ', 
'GUAIRA':'GUAÍRA', 
'MOSSUNGE':'MOSSUNGUÊ', 
'AHU':'AHÚ', 
'ALTO BARIGUI':'CIDADE INDUSTRIAL DE CURITIBA', 
'VILA VERDE II':'CIDADE INDUSTRIAL DE CURITIBA', 
'VILA VERDE - CIC':'CIDADE INDUSTRIAL DE CURITIBA', 
'VILA SAO JOSE':'AUGUSTA', 
'ALTO BOQUEIRAO':'ALTO BOQUEIRÃO', 
'VILA GUAIRA':'GUAÍRA', 
'VILA AUTÓDROMO':'CAJURU', 
'VILA AUDI':'UBERABA', 
'VILA - TRINDADE':'CAJURU', 
'CAPAO DA IMBUIA':'CAPÃO DA IMBUIA', 
'PORTAO':'PORTÃO', 
'LINDOIA':'LINDÓIA', 
'PRANCHA 80.0':'PORTÃO', 
'CAPÃO DAIMBUIA':'CAPÃO DA IMBUIA', 
'S/N':'TIRAR FORA', 
'SABARA':'CIDADE INDUSTRIAL DE CURITIBA', 
'CAPÃO':'CAPÃO DA IMBUIA', 
'BOQUEIRAO':'BOQUEIRÃO', 
'SÃO DOMINGOS':'TIRAR FORA', 
'CAPAO RASO':'CAPÃO RASO', 
'None':'TIRAR FORA', 
'INDICAÇÕES CANCELADAS':'TIRAR FORA', 
'JD. ITALIA':'SANTA FELICIDADE', 
'MORADIAS ARAGUAIA':'CAPÃO DA IMBUIA', 
'MORADIAS CJURU':'CAJURU', 
'VILA SÃO DOMINGOS':'CAJURU', 
'STA CANDIDA':'SANTA CÂNDIDA', 
'VILA AGRICOLA':'CAJURU', 
'JARDIM GABINETO':'CIDADE INDUSTRIAL DE CURITIBA', 
'UBERABA DE CIMA':'UBERABA', 
'ALTO DA XV':'ALTO DA RUA XV', 
'VILA VERDE CIC':'CIDADE INDUSTRIAL DE CURITIBA', 
'CAJURU (VILA OFICINAS)':'CAJURU', 
'C. COMPRIDO':'CAMPO COMPRIDO', 
'MARUMBU II (CAJURU)':'CAJURU', 
'AUGUSTA B':'AUGUSTA', 
'CENTENARIO':'CAJURU', 
'C SANTANA':'CAMPO DE SANTANA', 
'VILA CENTENÁRIO':'CAJURU', 
'MORADIAS CAJURU (CAJURU)':'CAJURU', 
'C DA IMBUIA':'CAPÃO DA IMBUIA', 
'JARDIM FLORIANÓPOLIS':'CAJURU', 
'PANTANAL':'ALTO BOQUEIRÃO', 
'VILA TRINDADE':'CAJURU', 
'JARDIM CABINETO':'CIDADE INDUSTRIAL DE CURITIBA', 
'PLANTA EVARISTO (UBERABA)':'UBERABA'}
    if nome_corrigir in dict_correcoes:
        nome = dict_correcoes[nome_corrigir]
        
    else:
        nome = nome_corrigir
    
    return nome

# Programa em si

In [7]:
df_alvaras_raw = Base_Alvaras()
gdf_mapa_bairros = Mapa_Curitiba_Bairros()

C:\Users\User\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
df_alvaras = df_alvaras_raw.copy()

## Tratamento básico do DataFrame

In [9]:
# Verificação geral dos dados:
df_alvaras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515876 entries, 0 to 515875
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   nome_empresarial              515876 non-null  object 
 1   data_inicio_atividade         515874 non-null  object 
 2   numero_do_alvara              515876 non-null  int64  
 3   nome_da_empresa               1895 non-null    object 
 4   data_emissao                  515871 non-null  object 
 5   data_expiracao                47104 non-null   object 
 6   atividade_principal           515876 non-null  object 
 7   atividade_secundaria1         208950 non-null  object 
 8   atividade_secundaria2         131217 non-null  object 
 9   endereco                      515876 non-null  object 
 10  numero                        515763 non-null  object 
 11  unidate                       314288 non-null  object 
 12  andar                         95147 non-null

In [10]:
# Retirada das colunas 'cidade' e 'estado', já nulas
# Retirada das colunas 'latitude', 'longitude' e 'localidade', pois se utilizará apenas a granularidade de bairros e regionais
# Data de expiração nula: descarte dessa coluna para a análise
df_alvaras = df_alvaras.drop(columns = {'cidade', 'estado', 'latitude', 'longitude', 'localidade', 'data_expiracao'})

In [11]:
# Início de atividade nula: considerados como "data_emissao":
df_alvaras.loc[df_alvaras['data_inicio_atividade'].isna(), 'data_inicio_atividade'] = df_alvaras['data_emissao']

# Nomes de empresa nulos: considerados como "nome_empresarial":
df_alvaras.loc[df_alvaras['nome_da_empresa'].isna(), 'nome_da_empresa'] = df_alvaras['nome_empresarial']

In [12]:
# Filtro das datas: apenas alvarás emitidos entre 2013 e 2018:
df_alvaras = df_alvaras.loc[(df_alvaras['data_emissao'] >= dt.date(2013,1,1)) & (df_alvaras['data_emissao'] <= dt.date(2018,12,31))].reset_index(drop=True)

In [13]:
# Limpezas dos nomes dos bairros, para que estejam iguais à base do mapa
gdf_mapa_bairros['nome'] = gdf_mapa_bairros['nome'].str.replace('CAPÃO DA IMBÚIA', 'CAPÃO DA IMBUIA')
df_alvaras['bairro'] = df_alvaras['bairro'].str.upper()
df_alvaras['bairro'] = df_alvaras.apply(lambda x: Corrige_Nomes_Bairros(x['bairro']), axis=1)
df_alvaras = df_alvaras.loc[df_alvaras.bairro != 'TIRAR FORA'].reset_index(drop=True)

## Agrupamentos básicos

### Análise de mapas e Entropia de Shannon